In [5]:
import os
import nltk
from nltk import word_tokenize
import pandas as pd
import re
from nltk.corpus import stopwords
import string
from nltk import word_tokenize as wt
from nltk import wordnet as wn


In [24]:
# Objective
'''
Use the NLTK package to try to better identify the industries and wrongful acts present in the majority of the text.  
Then use this information as the basis for the dictionary that you will create to categorize each text by industry and 
wrongful act. 
'''

'\nUse the NLTK package to try to better identify the industries and wrongful acts present in the majority of the text.  \nThen use this information as the basis for the dictionary that you will create to categorize each text by industry and \nwrongful act. \n'

In [6]:
# Import Concatenaed Text

chdir = os.chdir(r'/Users/ccirelli2/Public/Python Programing Docs/Corpus Texts/')
File = open('EEOC Articles - Concatenated.txt')
Text = File.read()


In [ ]:
# Tokenize Text
Text_tokenized = word_tokenize(Text)

In [50]:
# Import Cleaning Packages
chdir = os.chdir(r'/Users/ccirelli2/Public/Python Programing Docs/Modules')

def create_dict_punct():
    import string
    Dict = {}
    Punct = string.punctuation
    for x in Punct:
        Dict[x] = ''
    return Dict 

def create_dict_stopwords():
    from nltk.corpus import stopwords
    Stopwords = stopwords.words('english')                  
    Dict = {}
    for x in Stopwords:
        Dict[x] = ''
    return Dict

def strip_punctuation(Token_list):
    dict_punct = create_dict_punct()
    List = []
    for x in Token_list:
        if x not in dict_punct:
            List.append(x)
    return List
    
def strip_stop_words(Token_list):
    stop_words = create_dict_stopwords()
    List = []
    for x in Token_list:
        if x not in stop_words:
            List.append(x)
    return List

def strip_tokens_forwardSlash_x2(Token_list):
    List = []
    for token in Token_list:
        if '\\' not in token and '/' not in token:
            
            List.append(token)
    return List

def strip_two_variable_tokens(Token_list):
    List = [x for x in Token_list if len(x) > 2]
    return List

def get_isalpha(Token_list):
    List = [x for x in Token_list if x.isalpha()]
    return List

def get_unusual_english_words(Token_list):
    list_vocab = set(w.lower() for w in Token_list if w.isalpha())
    english_vocab = set(w.lower() for w in nltk.corpus.words.words())
    unusual = list_vocab.difference(english_vocab)  # unclear if 'difference' is being call on a standard list obj
    return sorted(unusual)

def get_spanish_words_in_text(Token_list):
    list_vocab = set(w.lower() for w in Token_list if w.isalpha())
    english_vocab = set(w.lower() for w in nltk.corpus.cess_esp.words())
    list_spanish_words = list_vocab.difference(english_vocab)  # unclear if 'difference' is being call on a standard list obj
    return sorted(unusual)

def create_spanish_dictionary():
    Spanish_word_dict = {}
    for x in nltk.corpus.cess_esp.words():
        Spanish_word_dict[x] = ''
    return Spanish_word_dict

def strip_spanish_words(Token_list):
    Spanish_word_dict = create_spanish_dictionary()
    List = []
    for x in Token_list:
        if x not in Spanish_word_dict:
            List.append(x)
    return List

def get_synsets(List_tokens):
    List = []
    for token in List_tokens:
        Synset = nltk.corpus.wordnet.synsets(token)
        if Synset == []:
            List.append(token)
        else:
            Synset = Synset[0]
            List.append(Synset)
    return List

def word_count_frequency_table(Text):
    Dict = {}
    
    for x in Text:
        if x in Dict.keys():
            Dict[x] = Dict[x] + 1
        else:
            Dict[x] = 1
    
    df = pd.DataFrame(Dict, index = [1])
    df_tran = pd.DataFrame.transpose(df)
    df_final = df_tran.sort_values(1, ascending = False)
    df_rename = df_final.rename(columns = {1:'Word Count'})
    return df_rename

In [1]:
# Create Cleaning Pipeline

def text_cleaning_pipeline(Text):
    # Convert text to lowercase
    Text = [x.lower() for x in Text]  
    # Strip punctuation, stopwords, words contaning //
    Text_strip_punct = strip_punctuation(Text)
    Text_strip_stop_words = strip_stop_words(Text_strip_punct)
    Text_strip_forwardslash = strip_tokens_forwardSlash_x2(Text_strip_stop_words)
    Text_strip_two_variable_tokens = strip_two_variable_tokens(Text_strip_forwardslash)
    Text_get_isalpha = get_isalpha(Text_strip_two_variable_tokens)
    Text_strip_spanish = strip_spanish_words(Text_get_isalpha)
    return Text_strip_spanish
    
Clean_text = text_cleaning_pipeline(Text_tokenized)





NameError: name 'Text_tokenized' is not defined

In [29]:
print('Num words original text', len(Text_tokenized))
print('Num words cleaned text', len(Clean_text))
print('')


print('Length set of original text', len(Text_tokenized))
print('Length set of cleaned text', len(set(Clean_text)))

Num words original text 2284787
Num words cleaned text 779166

Length set of original text 2284787
Length set of cleaned text 18391


In [30]:
# Create Synset List

Synsets_list = get_synsets(Clean_text)


In [31]:
print(len(Synsets_list))

779166


In [32]:
def clean_synset(Synset_list):
    Synset_clean_list = []
    
    # Regex Exp - Find
    Regex = "[a-z]+\.[a-z]\.[0-9]+"
    Re_compile = re.compile(Regex)
    
    # Regex loop expression to obtain synset
    for synset in Synset_list:
        
        # Search for Regex Exp
        synset = str(synset)
        Re_search = re.search(Re_compile, synset)
        
        if Re_search == None:
            Synset_clean_list.append(synset)
        
        elif '(' not in synset:
            Synset_clean_list.append(synset)
        
        else:
            Re_group = Re_search.group()
            Re_split = Re_group.split('.')
            Synset_clean_list.append(Re_split[0])
            
    # Return Clean Synset 
    return Synset_clean_list


Clean_synset_list = clean_synset(Synsets_list)


In [37]:
print('Length of Synset List => ', len(Clean_synset_list))



Length of Synset List =>  779166


In [ ]:
'''#### Stemming

# Base Object for Stemming
class nltk.stem.api.StemmerI[source]
Bases: object

A processing interface for removing morphological affixes from words. This process is known as stemming.
stem(token)[source]
Strip affixes from the token and return the stem.

# Regex Stemming
lass nltk.stem.regexp.RegexpStemmer(regexp, min=0)[source]
Bases: nltk.stem.api.StemmerI

A stemmer that uses regular expressions to identify morphological affixes. 
Any substrings that match the regular expressions will be removed.


# Wordnet

class nltk.stem.wordnet.WordNetLemmatizer[source]
Bases: object

WordNet Lemmatizer
Lemmatize using WordNet’s built-in morphy function. 
Returns the input word unchanged if it cannot be found in WordNet.

'''



In [80]:
# Get Stem Clean Synset List

from nltk.stem.porter import *

def get_stem_of_token(Token_list):
    stemmer = PorterStemmer()
    Stem_list = []
    
    for x in Token_list:
        stem = stemmer.stem(x)
        Stem_list.append(stem)
        
    return Stem_list

Stem_list = get_stem(Clean_synset_list)


In [83]:
# Frequency Table

wc_freq_table = word_count_frequency_table(Stem_list)




In [84]:
chdir = os.chdir(r'/Users/ccirelli2/Public/Python Programing Docs/Projects/Web Scraping Project - EEOC Articles')


def write_to_excel(dataframe, filename):
    import pandas as pd
    writer = pd.ExcelWriter(filename+'.xlsx')
    dataframe.to_excel(writer, sheet_name = 'Data')
    writer.save()
    
write_to_excel(wc_freq_table, 'EEOC_Articles_Synset_freq_table')
    